## <center> Аттестация. Практическое задание в Jupyter

В данном блоке вам предстоит решить задачу, которую автор когда-то решал на одном из своих собеседований.
Вам предоставлена таблица с данными (скачать файл можно на платформе) о ежедневной выработке энергии солнечными батареями. Информация в ней содержится в двух столбцах: 
* date - дата наблюдения
* cum_power - накопленная (кумулятивная) выработка энергии на указанный день

То есть перед вами классический пример временного ряда: числовой показатель, зависящий от времени. Давайте с ним поработаем.


In [13]:
import pandas as pd
import numpy as np
import plotly.express as px

In [14]:
df = pd.read_csv('solarpower.zip')
df.head()

,date,cum_power
0,2017-01-01,20111.0
1,2017-01-02,20112.0
2,2017-01-03,20115.0
3,2017-01-04,20117.0
4,2017-01-05,20119.0


1. В нашем временном ряде есть несколько пропусков, они обозначены числом -1. Заполните пропуски средним арифметическим между двумя соседними наблюдениями: </p>
    $$x^*_i  = \frac{x_{i-1} + x_{i+1}}{2}$$ 
    
Совет: проще всего будет сделать это, используя цикл по индексам элементов столбца cum_power, но вы можете предложить более изящное решение.


In [15]:
display(df[df['cum_power'] == -1]) # найдем все пропуски
for i in range(df.shape[0]):
    if df.iloc[i, 1] == -1:
        df.iloc[i,1] = ( df.iloc[i-1,1] + df.iloc[i+1,1]) / 2
(df[df['cum_power'] == -1]).count() # проверим отсутствие пропусков

,date,cum_power
45,2017-02-15,-1.0
647,2018-10-10,-1.0


date         0
cum_power    0
dtype: int64

2. Теперь, когда таблица заполнена, создайте столбец day_power - ежедневный прирост выработки солнечной энергии.

    После чего создайте данные о ежемесячной выработке энергии на каждый год. 
    
Совет: для того, чтобы выделить из даты год и месяц проще всего воспользоваться методом аксессора dt.to_period('M'), который позволяет выделять периоды, в данном случае 'М' - месяц.

In [16]:
df['day_power'] = df['cum_power'].diff() #ежедневный прирост выработки солнечной энергии

In [17]:
df['date'] = pd.to_datetime(df['date']) #переводим формат признака в "datetime"

month_power_df = df.groupby(df['date'].dt.to_period('m')).sum() #выделяем период "месяцы"
month_power_df.index = month_power_df.index.to_series().astype(str) #меняем формат даты обратно на "строковый"

month_power_df  #получили сгруппированные данные по месяцам

,cum_power,day_power
date,,
2017-01,624746.0,97.0
2017-02,567197.0,104.0
2017-03,633782.0,334.0
2017-04,626551.0,489.0
2017-05,663651.0,589.0
2017-06,662047.0,645.0
2017-07,702680.0,562.0
2017-08,718539.0,467.0
2017-09,707203.0,346.0


3. Постройте график временного ряда - ежемесячной выработки солнечной энергии. Не забудьте подписать оси и сам график.
Укажите в какие сезоны наблюдается подъем, а в какие спад?

In [18]:
fig = px.line(
    month_power_df,
    y='day_power',
    title='Ежемесячная выработки солнечной энергии'
)
fig.update_layout(
    xaxis_title='Месяц и год',
    yaxis_title='Выработк солнечной энергии'
)
fig.show()

<center> <img src="https://github.com/RuslanMavlitov/sf_data_science/raw/main/SF_Python_attestation/newplot.png", width=900, heigth=500>   

Анализ графика: Выработка солнечной энергии сильно зависит от количества солнечных дней (Л - логика), поэтому отчетливо видно увеличении выработки в летний период и снижение в зимний. 
Аномалия: Июнь 2018 года имеет нестандартный провал, хотя ожидался максимум выработки. Погуглим... ничего не находим, так как незнаем где географически находятся наша солнечная станция. Возможно был плановый ремонт оборудования или внештатный частичный вывод оборудования из работы.
